In [1]:
%matplotlib qt5

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.gridspec as gs
import scipy as sp
import os
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from igraph import Graph
import ipdb

rd = np.random
dst = sp.spatial.distance
non_zero_force = 0

# Defining Functions

In [2]:
def charge_G(file_path):
    if file_path[-1] == "f":
        g = nx.read_gpickle(file_path)
        
    if file_path[-1] == "t":
        g = Graph.Read_Pickle(file_path)
        g.to_undirected()
    return g

# Placing Initial Condictions

In [3]:
g_nx = charge_G('../../data/net/net_nx_cart_cust.gexf')
g_nx = max(nx.connected_component_subgraphs(g_nx), key=len)
g_ig = charge_G('../../data/net/net_ig_cart_cust.dat')
g_ig = g_ig.clusters().giant()

alphabet = list(map(chr, range(97, 123)))

beta = 3
alpha = 2
L = 3

# Getting Position Attributes

In [4]:
G_nodes = np.array(list(g_nx.nodes))
G_pos = np.array(
    [list(x) for x in nx.get_node_attributes(g_nx, 'n_pos').values()])

#--------------------------Get m attribute-----------------------------------

D_norm_m = G_pos.copy()[:, 0]
P_norm_m = beta*np.exp(-D_norm_m/(L*alpha))
k_norm_m = dict(zip(G_nodes, P_norm_m))
nx.set_node_attributes(g_nx, name='k_norm_m', values=k_norm_m)

D_inv_m = np.repeat(D_norm_m.max(), G_nodes.size) - D_norm_m
P_inv_m = beta*np.exp(-D_inv_m/(L*alpha))
k_inv_m = dict(zip(G_nodes, P_inv_m))
nx.set_node_attributes(g_nx, name='k_inv_m', values=k_inv_m)


p_norm_m = [x/P_norm_m.sum() for x in P_norm_m]
p_inv_m = [x/P_inv_m.sum() for x in P_inv_m]


#--------------------------Get d attribute-----------------------------------

D_norm_d = G_pos.copy()[:, 1]
P_norm_d = beta*np.exp(-D_norm_d/(L*alpha))
k_norm_d = dict(zip(G_nodes, P_norm_d))
nx.set_node_attributes(g_nx, name='k_norm_d', values=k_norm_d)

D_inv_d = np.repeat(D_norm_d.max(), G_nodes.size) - D_norm_d
P_inv_d = beta*np.exp(-D_inv_d/(L*alpha))
k_inv_d = dict(zip(G_nodes, P_inv_d))
nx.set_node_attributes(g_nx, name='k_inv_d', values=k_inv_d)

p_norm_d = [x/P_norm_d.sum() for x in P_norm_d]
p_inv_d = [x/P_inv_d.sum() for x in P_inv_d]


#--------------------------Get a attribute-----------------------------------

D_norm_a = G_pos.copy()[:, 2]
P_norm_a = beta*np.exp(-D_norm_a/(L*alpha))
k_norm_a = dict(zip(G_nodes, P_norm_a))
nx.set_node_attributes(g_nx, name='k_norm_a', values=k_norm_a)

D_inv_a = np.repeat(D_norm_a.max(), G_nodes.size) - D_norm_a
P_inv_a = beta*np.exp(-D_inv_a/(L*alpha))
k_inv_a = dict(zip(G_nodes, P_inv_a))
nx.set_node_attributes(g_nx, name='k_inv_a', values=k_inv_a)

p_norm_a = [x/P_norm_a.sum() for x in P_norm_a]
p_inv_a = [x/P_inv_a.sum() for x in P_inv_a]

# Plotting Position Attributes

In [11]:
fig, axs = plt.subplots(nrows=2, ncols=3, constrained_layout=True)

#ax1.scatter(D_norm_m, [100*i for i in p_norm_d])
#ax2.scatter(D_norm_m, [100*i for i in  p_inv_d])

axs[0,0].scatter(D_norm_m, [100*i for i in p_norm_m])
axs[1,0].scatter(D_norm_m, [100*i for i in  p_inv_m])

axs[0,0].get_shared_x_axes().join(axs[0,0], axs[1,0])
axs[0,0].set_xticklabels([])

axs[0,0].set_ylabel("normal prob.\n")
axs[1,0].set_ylabel("inverse prob.")
axs[1,0].set_xlabel("Distance")
axs[0,0].set_title("midl\n(a)")
axs[1,0].set_title("(d)")



axs[0,1].scatter(D_norm_d, [100*i for i in p_norm_d])
axs[1,1].scatter(D_norm_d, [100*i for i in  p_inv_d])

axs[0,1].get_shared_x_axes().join(axs[0,1], axs[1,1])
axs[0,1].set_xticklabels([])

axs[1,1].set_xlabel("Distance")
axs[0,1].set_title("dist\n(b)")
axs[1,1].set_title("(e)")



axs[0,2].scatter(D_norm_a, [100*i for i in p_norm_a])
axs[1,2].scatter(D_norm_a, [100*i for i in  p_inv_a])

axs[0,2].get_shared_x_axes().join(axs[0,2], axs[1,2])
axs[0,2].set_xticklabels([])

axs[1,2].set_xlabel("Distance")
axs[0,2].set_title("angl\n(c)")
axs[1,2].set_title("(f)")


#ax1.set_ylim(-0.05, 0.02)
#ax2.set_ylim(-0.05, 0.02)

plt.show()

# Getting Network Attributes

In [6]:
#-------------NetworkX Methods----------------------------
G_degree = np.array(list(g_nx.degree))[:, 1]+non_zero_force
p_degree = np.array([x/np.sum(G_degree) for x in G_degree])

G_cluster = np.array(
    np.array(list(nx.clustering(g_nx).values())))+non_zero_force
p_cluster = np.array([x/np.sum(G_cluster) for x in G_cluster])

#-------------iGraph Methods----------------------------
G_betweenness = np.nan_to_num(
    np.array(g_ig.betweenness()))+non_zero_force
p_betweenness = np.array([x/np.sum(G_betweenness) for x in G_betweenness])

G_closeness = np.nan_to_num(
    np.array(g_ig.closeness()))+non_zero_force
p_closeness = np.array([x/np.sum(G_closeness) for x in G_closeness])

G_eigenvector = np.nan_to_num(np.array(
    g_ig.eigenvector_centrality()))+non_zero_force
p_eigenvector = np.array([x/np.sum(G_eigenvector) for x in G_eigenvector])

# Plotting Position Attributes

In [42]:
fig, axs = plt.subplots(nrows=5, ncols=3, constrained_layout=True, figsize=[10,10])

axs[0,0].scatter(D_norm_m, G_degree, s=2, alpha=0.1)
axs[0,1].scatter(D_norm_d, G_degree, s=2, alpha=0.1)
axs[0,2].scatter(D_norm_a, G_degree, s=2, alpha=0.1)

axs[1,0].scatter(D_norm_m, G_cluster, s=2, alpha=0.1)
axs[1,1].scatter(D_norm_d, G_cluster, s=2, alpha=0.1)
axs[1,2].scatter(D_norm_a, G_cluster, s=2, alpha=0.1)

axs[2,0].plot(D_norm_m, G_betweenness, 'o', markersize=2, alpha=0.1)
axs[2,1].plot(D_norm_d, G_betweenness, 'o', markersize=2, alpha=0.1)
axs[2,2].plot(D_norm_a, G_betweenness, 'o', markersize=2, alpha=0.1)
axs[2,0].set_yscale('log')
axs[2,1].set_yscale('log')
axs[2,2].set_yscale('log')

axs[3,0].scatter(D_norm_m, G_closeness, s=2, alpha=0.1)
axs[3,1].scatter(D_norm_d, G_closeness, s=2, alpha=0.1)
axs[3,2].scatter(D_norm_a, G_closeness, s=2, alpha=0.1)

axs[4,0].plot(D_norm_m, G_eigenvector, 'o', markersize=2, alpha=0.1)
axs[4,1].plot(D_norm_d, G_eigenvector, 'o', markersize=2, alpha=0.1)
axs[4,2].plot(D_norm_a, G_eigenvector, 'o', markersize=2, alpha=0.1)
axs[4,0].set_yscale('log')
axs[4,1].set_yscale('log')
axs[4,2].set_yscale('log')

axs[0,0].get_shared_x_axes().join(axs[0,0], axs[1,0])
axs[1,0].get_shared_x_axes().join(axs[1,0], axs[2,0])
axs[2,0].get_shared_x_axes().join(axs[2,0], axs[3,0])
axs[3,0].get_shared_x_axes().join(axs[3,0], axs[4,0])

axs[0,1].get_shared_x_axes().join(axs[0,1], axs[1,1])
axs[1,1].get_shared_x_axes().join(axs[1,1], axs[2,1])
axs[2,1].get_shared_x_axes().join(axs[2,1], axs[3,1])
axs[3,1].get_shared_x_axes().join(axs[3,1], axs[4,1])

axs[0,2].get_shared_x_axes().join(axs[0,2], axs[1,2])
axs[1,2].get_shared_x_axes().join(axs[1,2], axs[2,2])
axs[2,2].get_shared_x_axes().join(axs[2,2], axs[3,2])
axs[3,2].get_shared_x_axes().join(axs[3,2], axs[4,2])

axs[0,0].set_xticklabels([])
axs[1,0].set_xticklabels([])
axs[2,0].set_xticklabels([])
axs[3,0].set_xticklabels([])

axs[0,1].set_xticklabels([])
axs[1,1].set_xticklabels([])
axs[2,1].set_xticklabels([])
axs[3,1].set_xticklabels([])

axs[0,2].set_xticklabels([])
axs[1,2].set_xticklabels([])
axs[2,2].set_xticklabels([])
axs[3,2].set_xticklabels([])

axs[0,1].set_yticklabels([])
axs[1,1].set_yticklabels([])
axs[2,1].set_yticklabels([])
axs[3,1].set_yticklabels([])
axs[4,1].set_yticklabels([])

axs[0,2].set_yticklabels([])
axs[1,2].set_yticklabels([])
axs[2,2].set_yticklabels([])
axs[3,2].set_yticklabels([])
axs[4,2].set_yticklabels([])

axs[0,0].set_ylabel("Degree")
axs[1,0].set_ylabel("Cluster")
axs[2,0].set_ylabel("Betweenness")
axs[3,0].set_ylabel("Closeness")
axs[4,0].set_ylabel("Eigenvector")

# axs[0,0].set_ylabel("Deg")
# axs[1,0].set_ylabel("Clu")
# axs[2,0].set_ylabel("Bet")
# axs[3,0].set_ylabel("Clo")
# axs[4,0].set_ylabel("Eig")

axs[4,0].set_xlabel("Distance")
axs[4,1].set_xlabel("Distance")
axs[4,2].set_xlabel("Distance")

mark = 0
for r, linha in enumerate(axs):
    for c, graph in enumerate(linha):
        # if r == 0:
        #     if c == 0:
        #         axs[r, c].set_title('middline\n({})'.format(alphabet[mark]))
        #     elif c == 1:
        #         axs[r, c].set_title('distance\n({})'.format(alphabet[mark]))
        #     elif c == 2:
        #         axs[r, c].set_title('angle\n({})'.format(alphabet[mark]))
        # else:
        #     axs[r, c].set_title('({})'.format(alphabet[mark]))
        axs[r, c].set_title('({})'.format(alphabet[mark]))
        mark += 1

plt.savefig('../../results/properties-distribution/net_distr_lab_noname.jpg')
plt.savefig('../../results/properties-distribution/net_distr_lab_noname.eps')
plt.savefig('../../results/properties-distribution/net_distr_lab_noname.pdf')